In [10]:
import csv
def get_data(file_path, train):
    csvfile = open(file_path, 'r')
    data = [each for each in csv.reader(csvfile, delimiter = '#')]
    """    
    表格中的数据按照#来划分。
    第一部分是表及其别名#第二部分是连接条件
    （有的不存在）#第三部分是where子句，不同条件之间用,分隔#最后是选择数目
    """
    for i in range(len(data)):
        tables = data[i][0].split(',')
        for j in range(len(tables)):
            tables[j] = tables[j].split(' ')[1]
        data[i][0] = tables 
        #第一个部分是要选的表, 存的是别名

        #第二个部分是连接条件,没有的话就是''
        joins = []
        if len(data[i][0]) > 1:#按照要求是只有两个表连接,但是好像有看到多个表连接的
            join_cons = data[i][1].split(',')
            for j in range(len(join_cons)):
                join_cons[j] = join_cons[j].split('=')#连接条件应该只能是等于
                joins.append(join_cons[j])
        data[i][1] = joins

        #第三个部分是条件 #应该是每3个为一条，还得识别属性
        #之后从Column_min_max_vals.csv里读一下再分一下属性,给属性编号
        items = data[i][2].split(',')
        conditions = []
        for j in range(int(len(items) / 3)):
            condition = []
            condition.append(items[3 * j])
            condition.append(items[3 * j + 1])
            condition.append(int(items[3 * j + 2])) #不知道会不会有其他类型的,float什么的
            conditions.append(condition)
        data[i][2] = conditions

        #第四个部分是查询结果数目
        if train == True:
            data[i][3] = int(data[i][3])
    return data

'''data的示例:
[['t', 'mi_idx'], [['t.id', 'mi_idx.movie_id']], [['t.kind_id', '=', 7], ['mi_idx.info_type_id', '>', 99]], 283812]
[['t'], [], [['t.production_year', '>', 2004]], 1107925]
'''

"data的示例:\n[['t', 'mi_idx'], [['t.id', 'mi_idx.movie_id']], [['t.kind_id', '=', 7], ['mi_idx.info_type_id', '>', 99]], 283812]\n[['t'], [], [['t.production_year', '>', 2004]], 1107925]\n"

In [11]:
example_list=[]
#感觉是回归分析
#把自变量x和因变量y分开
train_data = get_data("C://Users//Linnea_keke//Desktop//大数据计算智能//ruccardinality//training_data.csv", True)

for i in range(20):
    example_list.append(train_data[i])
    print(example_list[i])
#这个后面要注释掉，拿出出20条数据用于调试和观看结果

train_data_x = []
train_data_y = []
for i in range(len(train_data)):
    train_data_x.append(train_data[i][0:3])
    train_data_y.append(train_data[i][3])
    

[['t', 'mi_idx'], [['t.id', 'mi_idx.movie_id']], [['t.kind_id', '=', 7], ['mi_idx.info_type_id', '>', 99]], 283812]
[['t'], [], [['t.production_year', '>', 2004]], 1107925]
[['mi'], [], [['mi.info_type_id', '<', 4]], 3624977]
[['t', 'mc'], [['t.id', 'mc.movie_id']], [['mc.company_id', '<', 27]], 134807]
[['mk'], [], [['mk.keyword_id', '<', 55]], 54826]
[['t', 'mc', 'mi'], [['t.id', 'mc.movie_id'], ['t.id', 'mi.movie_id']], [['t.production_year', '>', 1977], ['mc.company_id', '>', 71403], ['mi.info_type_id', '<', 4]], 1172762]
[['mk'], [], [['mk.keyword_id', '<', 35049]], 4007119]
[['t', 'mc'], [['t.id', 'mc.movie_id']], [], 2609129]
[['mi_idx'], [], [['mi_idx.info_type_id', '>', 99]], 920110]
[['t', 'mi', 'mi_idx'], [['t.id', 'mi.movie_id'], ['t.id', 'mi_idx.movie_id']], [['mi.info_type_id', '>', 16], ['mi_idx.info_type_id', '=', 100]], 1603946]
[['t', 'ci', 'mk'], [['t.id', 'ci.movie_id'], ['t.id', 'mk.movie_id']], [['ci.person_id', '=', 890821], ['ci.role_id', '=', 1], ['mk.keyword_i

In [12]:
import pandas as pd
column_details = pd.read_csv("C://Users//Linnea_keke//Desktop//大数据计算智能//ruccardinality//column_min_max_vals.csv") #最小值、最大值、元组数目、不重复记录数目

#给每个属性一个编号，对应在自变量x的位置
feature_index = {}
for i in range(len(column_details)):
    feature_index[column_details.iloc[i]['name']] = i
    #从0开始编，可以在向量中用向量的下标除以2就能对应到属性
    #生成的数据是字典格式

#给每个表一个编号，数据中总共只有5个表
table_index = {'t' : 0, 'mc' : 1, 'ci' : 2, 'mi' : 3, 'mi_idx' : 4, 'mk' : 5}
print(column_details)
print(feature_index.get('t.kind_id'))
print(table_index.get('mk'))#使用字典的方式

                   name   min       max  cardinality  num_unique_values
0                  t.id     1   2528312      2528312            2528312
1             t.kind_id     1         7      2528312                  6
2     t.production_year  1880      2019      2528312                133
3                 mc.id     1   2609129      2609129            2609129
4         mc.company_id     1    234997      2609129             234997
5           mc.movie_id     2   2525745      2609129            1087236
6    mc.company_type_id     1         2      2609129                  2
7                 ci.id     1  36244344     36244344           36244344
8           ci.movie_id     1   2525975     36244344            2331601
9          ci.person_id     1   4061926     36244344            4051810
10           ci.role_id     1        11     36244344                 11
11                mi.id     1  14835720     14835720           14835720
12          mi.movie_id     1   2526430     14835720            

In [13]:
#一共20个属性,对应编号0-19
#自变量x前40维,属性编号i的对应[2i,2i+1]
#对应存在查询语句里对这个属性限制的最小值和最大值和等值
#没有限制的话最小值和最大值从column_details里取
#没有选的表的属性对应的值设为一个不可能出现的值,预计-1? 还是所有没有限制的属性都设成-1?
#两个表的连接在40列后面再附上20列，20列表示20个属性，下标为40+i
#测试数据example_list=[]
#feature_index[column_details.iloc[i]['name']] = i
import numpy as np

join_array=[]
#从列表中提取属性名称，这里没有用到查询计划
def create_join_array(example):
    join=[]
    #比如传入example_list
    for i in range (len(example)):
        str1=example[i][1]
        X_temp=np.zeros(20)
         #X_temp为每一行的连接条件的初始化数组
        for j in range(len(str1)):
            temp=str1[j]
            #print(temp)        
            for k in range (len(temp)):
                temp_name=temp[k]                     
                num=feature_index.get(temp_name)
                X_temp[num]=1
        join.append(X_temp)
        #join=np.row_stack((join,X_temp))
        #np.append(join,X_temp)
        #print(X_temp)
    #print(join)
    return join

join_array=np.array(create_join_array(train_data_x))
print(join_array[0])
print(join_array.shape)
        

#print(x_vector)
#print(example_list[0][1])
#print(list_join)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(100000, 20)


In [14]:
select_array=[]
np.set_printoptions(suppress=True)
#按照[low,upper,low,upper.......]的格式
#[ [['t.kind_id', '=', 7], ['mi_idx.info_type_id', '>', 99]], 283812]
def create_select_array(example):
    select=[]
    #比如传入example_list
    for i in range (len(example)):
        str1=example[i][2]
        X_temp=np.zeros(40)
         #X_temp为每一行的选择条件的初始化数组
        for j in range(len(str1)):
            temp=str1[j]
            #print(temp)
            #每一个j对应一个选择条件 
            if temp[1]=='=':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2]=temp[2]
                X_temp[num*2+1]=temp[2]
            elif temp[1]=='>':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
               # print(num)
                X_temp[num*2]=temp[2]#最小值
                X_temp[num*2+1]=column_details.iloc[num]['max']#上界从column_details里面取
                #print(X_temp)               
            else :
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2+1]=temp[2]#最大值
                X_temp[num*2]=column_details.iloc[num]['min']#下界从column_details里面取
        select.append(X_temp)
    return select

#select_array=np.array(create_select_array(example_list))
select_array=np.array(create_select_array(train_data_x))
print(select_array[0:10])
print(select_array.shape)



[[     0.      0.      7.      7.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.     99.    113.      0.      0.
       0.      0.      0.      0.]
 [     0.      0.      0.      0.   2004.   2019.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.]
 [     0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      1.
       4.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.]
 [     

In [15]:
x_vector=[]

#前面分开写，分开调试，这里是最终的函数
def create_X_vector(example):
    X=[]
    #比如传入example_list
    for i in range (len(example)):
        str1=example[i][1]
        str2=example[i][2]
        X_temp=np.zeros(60)
         #X_temp为每一行的连接条件的初始化数组
        for j in range(len(str1)):
            temp1=str1[j]
            #print(temp)        
            for k in range (len(temp1)):
                temp_name=temp1[k]                     
                num=feature_index.get(temp_name)
                X_temp[num+40]=1
        for k in range(len(str2)):
            temp=str2[k]
            #每一个j对应一个选择条件 
            if temp[1]=='=':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2]=temp[2]
                X_temp[num*2+1]=temp[2]
            elif temp[1]=='>':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
               # print(num)
                X_temp[num*2]=temp[2]#最小值
                X_temp[num*2+1]=column_details.iloc[num]['max']#上界从column_details里面取
                #print(X_temp)               
            else :
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2+1]=temp[2]#最大值
                X_temp[num*2]=column_details.iloc[num]['min']#下界从column_details里面取
        X.append(X_temp)
    return X

x_vector=np.array(create_X_vector(train_data_x))
print(x_vector[0:10])
print(x_vector.shape)

[[     0.      0.      7.      7.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.     99.    113.      0.      0.
       0.      0.      0.      0.      1.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      1.      0.      0.      0.      0.]
 [     0.      0.      0.      0.   2004.   2019.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.]
 [     0. 

In [43]:
#另一种数据处理格式，把连接关系存下来编号，把编号存进去
#这里先用20个数据尝试了一下
connection={ }
tempx_vector1=[]

def count_connection(example):
    X=[]
    for i in range (len(example)):    
        str1=example[i][1]
        X_temp=np.zeros(2)
        for j in range(len(str1)):
            temp=str1[j]
            temp_name1=temp[0] 
            temp_name2=temp[1]
            num1=feature_index.get(temp_name1)
            num2=feature_index.get(temp_name2)
            list1 = [num1,num2]
            key = tuple(list1)
            #print(key)
            if key in connection:
                X_temp[j]=connection.get(key)
            else:
                value=len(connection)
                #print(value)
                #print(key)
                connection[key]=value+1
                X_temp[j]=connection.get(key)
        X.append(X_temp) 
    return X   
tempx_vector1=np.array(count_connection(example_list))
print(tempx_vector1)
print(tempx_vector1.shape)
print(connection)

[[1. 0.]
 [0. 0.]
 [0. 0.]
 [2. 0.]
 [0. 0.]
 [2. 3.]
 [0. 0.]
 [2. 0.]
 [0. 0.]
 [3. 1.]
 [4. 5.]
 [3. 5.]
 [1. 0.]
 [0. 0.]
 [4. 0.]
 [0. 0.]
 [3. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
(20, 2)
{(0, 15): 1, (0, 5): 2, (0, 12): 3, (0, 8): 4, (0, 18): 5}


In [48]:
x_vector1=[]

#前面分开写，分开调试，这里是最终的函数
def X_connection(example):
    X=[]
    #比如传入example_list
    for i in range (len(example)):
        str1=example[i][1]
        str2=example[i][2]
        X_temp=np.zeros(42)
         #X_temp为每一行的连接条件的初始化数组
        for j in range(len(str1)):
            temp1=str1[j]
            temp_name1=temp1[0] 
            temp_name2=temp1[1]       
            num1=feature_index.get(temp_name1)
            num2=feature_index.get(temp_name2)
            list1 = [num1,num2]
            key = tuple(list1)
            #print(key)
            if key in connection:
                X_temp[40+j]=connection.get(key)
            else:
                value=len(connection)
                connection[key]=value+1
                X_temp[40+j]=connection.get(key) 
                
        #下面都是处理前40行
        for k in range(len(str2)):
            temp=str2[k]
            #每一个j对应一个选择条件 
            if temp[1]=='=':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2]=temp[2]
                X_temp[num*2+1]=temp[2]
            elif temp[1]=='>':
                temp_name=temp[0]
                num=feature_index.get(temp_name)
               # print(num)
                X_temp[num*2]=temp[2]#最小值
                X_temp[num*2+1]=column_details.iloc[num]['max']#上界从column_details里面取
                #print(X_temp)               
            else :
                temp_name=temp[0]
                num=feature_index.get(temp_name)
                X_temp[num*2+1]=temp[2]#最大值
                X_temp[num*2]=column_details.iloc[num]['min']#下界从column_details里面取
        X.append(X_temp)
    return X

x_vector1=np.array(X_connection(train_data_x))

print(x_vector1)
print(x_vector1.shape)
print(x_vector1[0:10])
print(connection)


[[    0.     0.     7. ...     0.     1.     0.]
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]
 ...
 [    0.     0.     7. ...     0.     3.     1.]
 [    0.     0.     7. ... 36449.     1.     5.]
 [    0.     0.     0. ...  3219.     0.     0.]]
(100000, 42)
[[     0.      0.      7.      7.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.     99.    113.      0.      0.
       0.      0.      0.      0.      1.      0.]
 [     0.      0.      0.      0.   2004.   2019.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.      0.      0.      0.      0.
       0.      0.      0.      0.      0.   

In [ ]:
"""
数据处理方式2
定义了函数X_connection(example)，example的地方传入train_data_x
处理测试数据的时候直接把test_data传进去，就能得到测试数据的X向量

处理好的测试数据的X向量叫 x_vector1
x_vector1 是100000*42维，前40列表示对大小值的约束，后面两列把连接关系编了个号，然后我看到最多只有三个表相连，那就有两个连接条件，所以设置成2维

目前10万条数据就找到了5种连接，编号分别为1,2,3,4,5
计算测试数据的X向量时，如果检测到不在字典中的属性，是可添加的，所以不用担心会错
"""

In [ ]:
#评测结果使用MSLE，评测分数越小代表结果越好
#(mean_squared_logarithmic_error):均方对数损失(mse的改进)

#预计可用的库：
#keras